In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001CE0D4D8310>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001CE0D4D96C0>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [2]:
os.environ["hf_token"]=os.getenv("hf_token")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\aswin\anaconda3\envs\langenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import PyPDFDirectoryLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
loader = PyPDFLoader("lawdocunment.pdf")
pages = loader.load()

In [5]:
pages[0]

Document(metadata={'producer': 'PyPDF', 'creator': 'Nitro Pro 12', 'creationdate': '', 'moddate': '2020-06-26T16:26:07+05:30', 'source': 'lawdocunment.pdf', 'total_pages': 171, 'page': 0, 'page_label': '1'}, page_content="KLE LAW ACADEMY BELAGAVI \n(Constituent Colleges: KLE Society’s Law College, Bengaluru, Gurusiddappa Kotambri Law College, \nHubballi, S.A. Manvi Law College, Gadag, KLE Society’s B.V. Bellad Law College, Belagavi, KLE Law \nCollege, Chikodi, and KLE College of Law, Kalamboli, Navi Mumbai) \n \n \nSTUDY MATERIAL \nfor \nCONTRACT I \nPrepared as per the syllabus prescribed by Karnataka State Law University (KSLU), Hubballi \n \nCompiled by \nDr. J.M. Mallikarjunaiah \nAnkita Pandey, Asst. Prof. \nKavyashree Bhandary, Asst. Prof. \n \n \nReviewed by \n \nDr. J.M. Mallikarjunaiah \nK.L.E. Society's Law College, Bengaluru \n \nThis study material is intended to be used as supplementary material to  the online classes and \nrecorded video lectures. It is prepared for the s

In [6]:
#### Spliitng the text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(pages)
vectorstore=Chroma.from_documents(documents=splits,embedding=embeddings)
retriever=vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001CE0AD2D9C0>, search_kwargs={})

In [7]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [9]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [12]:
response=rag_chain.invoke({"input":"What is contract"})
response

{'input': 'What is contract',
 'context': [Document(id='926d40b8-d88f-415e-9949-d08f6dbdc9b4', metadata={'creationdate': '', 'creator': 'Nitro Pro 12', 'moddate': '2020-06-26T16:26:07+05:30', 'page': 6, 'page_label': '7', 'producer': 'PyPDF', 'source': 'lawdocunment.pdf', 'total_pages': 171}, page_content='UNIT I \nIntroduction  \nThe Law of Contract constitutes the most important branch of mercantile or commercial law. \nIt affects everybody, more so, trade, commerce and industry. It may be said that the contract is \nthe foundation of the civilized world. The law relating to contract is governed by  the Indian \nContract Act, 1872 . The preamble to the Act says that it is an Act "to define and amend certain \nparts of the law relating to contract". It extends to the whole of Ind ia except the State of Jammu \nand Kashmir.   \nDefinition and Essentials of CONTRACT \nA Contract is an agreement enforceable by law. An agreement is enforceable by law, if it \nis made by the free consent o

In [13]:
response['answer']

'A contract is an agreement enforceable by law. It is made by the free consent of parties who are competent to contract, with a lawful object and for a lawful consideration, and is not hereby expressly declared to be void.'